In [14]:
import os
from dotenv import load_dotenv

In [15]:
sucess = load_dotenv(".venv/.env")

In [16]:
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")


In [2]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_ollama import ChatOllama

In [ ]:
llm = ChatOllama(temperature=0, model="gemma3:12b")
graph_transformer = LLMGraphTransformer(
    llm=llm
    )

In [69]:
text = """
Albert Einstein pronunciación en alemán: /ˈalbɐt ˈaɪnʃtaɪn/ (escucharⓘ);[2]​ (Ulm, Imperio alemán, 14 de marzo de 1879-Princeton, Estados Unidos, 18 de abril de 1955) fue un físico alemán de origen judío, nacionalizado después suizo, austriaco y estadounidense. Se le considera el científico más importante, conocido y popular del siglo XX.[3]​[4]​

En 1905, cuando era un joven físico desconocido, empleado en la Oficina de Patentes de Berna, publicó su teoría de la relatividad especial. En ella incorporó, en un marco teórico simple fundamentado en postulados físicos sencillos, conceptos y fenómenos estudiados antes por Henri Poincaré y Hendrik Lorentz. Como una consecuencia lógica de esta teoría, dedujo la ecuación de la física más conocida a nivel popular: la equivalencia masa-energía, E=mc². Ese año, publicó otros trabajos que sentarían algunas de las bases de la física estadística y de la mecánica cuántica.

En 1915, presentó la teoría de la relatividad general, en la que reformuló por completo el concepto de la gravedad.[5]​ Una de las consecuencias fue el surgimiento del estudio científico del origen y la evolución del universo por la rama de la física denominada cosmología. En 1919, cuando las observaciones británicas de un eclipse solar acontecido en mayo de ese año, confirmaron sus predicciones acerca de la curvatura de la luz, fue idolatrado por la prensa.[6]​ Einstein se convirtió en un ícono popular de la ciencia mundialmente famoso, un privilegio al alcance de muy pocos científicos.[7]​

Por sus explicaciones sobre el efecto fotoeléctrico y sus numerosas contribuciones a la física teórica, en 1921 obtuvo el Premio Nobel de Física y no por la Teoría de la Relatividad, pues el científico a quien se encomendó la tarea de evaluarla no la entendió, y temieron correr el riesgo de que luego se demostrase errónea.[8]​[9]​ En esa época era aún considerada un tanto controvertida.

Ante el ascenso del nazismo, Einstein abandonó Alemania en diciembre de 1932 con destino a Estados Unidos, donde se dedicó a la docencia en el Institute for Advanced Study. Se nacionalizó estadounidense en 1940. Durante sus últimos años trabajó por integrar en una misma teoría la fuerza gravitatoria y la electromagnética.

Aunque es considerado por algunos como el «padre de la bomba atómica», abogó por el federalismo mundial, el internacionalismo, el pacifismo, el sionismo y el socialismo democrático, con una fuerte devoción por la libertad individual y la libertad de expresión.[10]​[11]​[12]​[13]​ Fue proclamado «personaje del siglo XX» y el más preeminente científico por la revista Time.[14]​

"""

In [ ]:
documents = [Document(page_content=text)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Albert Einstein', type='Person', properties={}), Node(id='Ulm', type='City', properties={}), Node(id='Imperio Alemán', type='Country', properties={}), Node(id='14 De Marzo De 1879', type='Date', properties={}), Node(id='Princeton', type='City', properties={}), Node(id='Estados Unidos', type='Country', properties={}), Node(id='18 De Abril De 1955', type='Date', properties={}), Node(id='Físico', type='Profession', properties={}), Node(id='Alemán', type='Nationality', properties={}), Node(id='Judío', type='Religion', properties={}), Node(id='Suizo', type='Nationality', properties={}), Node(id='Austriaco', type='Nationality', properties={}), Node(id='Siglo Xx', type='Time period', properties={}), Node(id='Henri Poincaré', type='Person', properties={}), Node(id='Hendrik Lorentz', type='Person', properties={}), Node(id='E=Mc²', type='Equation', properties={}), Node(id='1915', type='Date', properties={}), Node(id='Relatividad General', type='Theory', properties=

In [ ]:
from langchain_neo4j import Neo4jGraph

In [ ]:
graph = Neo4jGraph(
    url=neo4j_url, 
    username=neo4j_user, 
    password=neo4j_password, 

)

In [76]:
graph_documents

[GraphDocument(nodes=[Node(id='Albert Einstein', type='Person', properties={}), Node(id='Ulm', type='City', properties={}), Node(id='Imperio Alemán', type='Country', properties={}), Node(id='14 De Marzo De 1879', type='Date', properties={}), Node(id='Princeton', type='City', properties={}), Node(id='Estados Unidos', type='Country', properties={}), Node(id='18 De Abril De 1955', type='Date', properties={}), Node(id='Físico', type='Profession', properties={}), Node(id='Alemán', type='Nationality', properties={}), Node(id='Judío', type='Religion', properties={}), Node(id='Suizo', type='Nationality', properties={}), Node(id='Austriaco', type='Nationality', properties={}), Node(id='Siglo Xx', type='Time period', properties={}), Node(id='Henri Poincaré', type='Person', properties={}), Node(id='Hendrik Lorentz', type='Person', properties={}), Node(id='E=Mc²', type='Equation', properties={}), Node(id='1915', type='Date', properties={}), Node(id='Relatividad General', type='Theory', properties=

In [77]:
graph.add_graph_documents(
    graph_documents, 
    baseEntityLabel=True,
    include_source=True
)

In [56]:
graph

In [57]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True,
    allow_dangerous_requests=True, # Required in recent versions to acknowledge Cypher execution
    enhanced_schema=True
)

response = chain.invoke({"query": "Explain me about Albert Einstein"})
print(response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Person {name: "Albert Einstein"})
RETURN n

Full Context:
[{'n': {'entity_type': 'Person', 'canonical_name': 'albert einstein', 'name': 'Albert Einstein', 'id': 'Albert Einstein'}}]

> Finished chain.
Albert Einstein is a person.


In [78]:
query = """
MATCH (n) 
DETACH DELETE n
"""
results = graph.query(query)
print(results)

[]
